# Workshop on Structure-Based Drug Design on a Laptop Spring 2026



# 6.1 Alchemical Relative Binding Free Energy Calculation

Based on  https://github.com/pablo-arantes/making-it-rain

In this session, we will calculate the relative binding free energy between two compounds for the HPK1 receptor. Under most conditions, the relative binding free energy $\Delta \Delta G_b$ is related to the ratios of IC50's or Ki's of the two compounds:

$\Delta \Delta G_b \simeq k_B T \ln \frac{Ki(B)}{Ki(A)} \simeq k_B T \ln \frac{IC50(B)}{IC50(A)}$

where $A$ and $B$ are the two ligands, and $k_B = 0.001987$ kcal/(mol⋅K) is Boltzmann's constant, T is the absolute temperature ($T=300 K$ here). We will use the [AToM-OpenMM](https://github.com/Gallicchio-Lab/AToM-OpenMM) software to conduct this calculation. This model works by placing one ligand into the binding site and the second as some distance away in the solvent. It then calculates the free energy of switching the positions of the two ligands, which is the relative binding free energy. Learn more about the AToM-OpenMM model [here](https://www.compmolbiophysbc.org/atom-openmm).  


To run the notebook:

1. Connect to a Google Colab Runtime by clicking on the down-arrow next to the `Connect` button on the upper left, select `Change runtime type`, and selecting a `T4` GPU. Then click `Connect`.
2. Execute the first cell to install `condacolab`. Google Colab will restart. When done, move to the following cells.
3. Execute the second cell to install the required software components then execute the third to test that the software is working.
4. At this point, you are at the `Create Working Directory and Upload Files` cell. Here enter the name of directory where the calculation will be performed. Any name that makes sense to you is fine. Avoid spaces.
    *  The cell will create the folder
    *  Using the file browser, navigate to the folder and upload the structure input files into it
    *  The workflow needs the pdb file of the protein-ligand complex we prepared in Maestro in Session 3.2
    *  It also needs the `.sdf` files of the two ligands already docked to the receptor.
5. In the following cell, named `Prepare system in a box of water` enter the names of the molecular input files, and the value of the displacement vector. The displacement vector specifies where in the solvent bulk the second ligand is placed relative to the first. The default should work by viewing the resulting system in the cell `View Protein-Ligand Complex in water`.
6. The next two cells display the structures of the two ligands with the atoms labeled by atom index. Pick three atoms of each ligand that will serve as "alignment" atoms to superimpose the two ligands. These atoms are usually picked in the rigid common core of the ligands. They cannot be co-linear.
7. Enter the indexes of the alignment atoms in the following cell called `Setup Relative Binding Free Energy Calculation`, then execute it to minimize, thermalize, equilibrate, and anneal the system. This will take 1/2 to 1 hour to complete.
8. The following cell runs the actual production calculation. It will take 4 hours to complete.
9. the cell `Analyisis: Binding free energy estimate` analyzes the output files generated during production to calculate the relative binding free energy. The first number of the output is the relative binding free energy estimate and the second its uncertainty in kcal/mol.
10. The final cell helps you download the simulation results to your laptop.


In [ ]:
#@title **Install Conda Colab**
#@markdown It will restart the kernel (session), don't worry.
# !pip install -q condacolab
# import condacolab
# condacolab.install()
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

In [ ]:
##@title **Install dependencies**
import subprocess
import sys

#fixes sys.path that gives torchvision import error
original_syspath = sys.path.copy()
print(original_syspath)
new_syspath = ['/content', '/env/python', '/usr/local/lib/python312.zip', '/usr/local/lib/python3.12', '/usr/local/lib/python3.12/lib-dynload', '/usr/local/lib/python3.12/site-packages' ]
sys.path = new_syspath
print(sys.path)

subprocess.run("mamba install openmm openmmforcefields setproctitle r-base nglview mdanalysis -y", shell=True)
subprocess.run("Rscript -e 'install.packages(\"UWHAM\", repos = \"http://cran.us.r-project.org\")'", shell=True)

#latest atom-openmm
subprocess.run("cd /content && git clone https://github.com/Gallicchio-Lab/AToM-OpenMM.git", shell=True)
subprocess.run("cd /content/AToM-OpenMM && pip install .", shell=True, capture_output=True, text=True)


In [ ]:
import openmm.testInstallation
openmm.testInstallation.main()

In [ ]:
#@title ### **Create Working Directory and Upload Files**
import os
workDir = '/content/fe_lig2' #@param {type:"string"}
os.makedirs(workDir, exist_ok=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ### **Prepare system in a box of water**
from atom_openmm.make_atm_system_from_rcpt_lig import make_system
from pathlib import Path

Protein_PDB_file_name = 'hpk1.pdb' #@param {type:"string"}
Ligand1_SDF_file_name = "lig28.sdf" #@param {type:"string"}
Ligand2_SDF_file_name = "lig30.sdf" #@param {type:"string"}
Displacement_vector_in_Angstroms = [0.0, 0.0, 45.0] #@param {type:"raw"}

rcpt_pdb = workDir + '/' + Protein_PDB_file_name
ligand1_sdf = workDir + '/' + Ligand1_SDF_file_name
ligand2_sdf = workDir + '/' + Ligand2_SDF_file_name

basename = Path(Protein_PDB_file_name).stem + \
  '-' + \
  Path(Ligand1_SDF_file_name).stem + \
  '-' + \
  Path(Ligand2_SDF_file_name).stem \

outxml = workDir + '/' + basename + '_sys.xml'
pdboutfile = workDir + '/' + basename + '.pdb'

make_system(
      receptorfile=rcpt_pdb,
      lig1sdffile=ligand1_sdf ,
      lig2sdffile=ligand2_sdf,
      displacement = Displacement_vector_in_Angstroms,
      xmloutfile=outxml,
      pdboutfile=pdboutfile,
      ionicstrength=0.15,
)


In [ ]:
#@title ### **View Protein-Ligand Complex in water**
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import nglview as nv
view = nv.show_structure_file(pdboutfile, default_representation = False)
view.add_cartoon('protein')
view.add_spacefill('ligand')
view.add_line('water')
view.add_ball_and_stick('ion')
view.layout.height = '800px'
view

In [ ]:
#@title ### **View Ligands to Select Alignment AToms**

import nglview as nv
view = nv.show_structure_file(ligand1_sdf, default_representation = False)
view.add_line(selection='all')
view.add_representation('label', selection='all', labelType="atomindex")
view

In [ ]:
view = nv.show_structure_file(ligand2_sdf, default_representation = False)
view.add_line(selection='all')
view.add_representation('label', selection='all', labelType="atomindex")
view

In [ ]:
#@title ### **Setup Relative Binding Free Energy Calculation**
import os
import openmm as mm
from openmm.app import *
from openmm import *
from openmm.unit import *
from multiprocessing import freeze_support
from atom_openmm.rbfe_structprep import rbfe_structprep
from atom_openmm.rbfe_production import rbfe_production


def get_indexes_from_query(topology, query):
    indexes = []
    for atom in topology.atoms():
        if eval(query):
            indexes.append(atom.index)
    indexes.sort()
    return indexes

def get_indexes_from_residue(residue, query = 'True'):
    indexes = []
    for atom in residue.atoms():
        if eval(query):
            indexes.append(atom.index)
    indexes.sort()
    return indexes

def cm_from_indexes(topology, positions, indexes):
    cm = Vec3(0,0,0)*nanometer
    n = 0
    for atom in topology.atoms():
        if atom.index in indexes:
            cm += positions[atom.index]
            n += 1
    cm = cm/float(n)
    return cm

options = {
        "JOB_TRANSPORT": 'LOCAL_OPENMM',
        "RE_SETUP": 'YES',
        "TEMPERATURES": [ 300.0 ],
        "LAMBDAS":      [0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00],
        "DIRECTION":    [   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
        "INTERMEDIATE": [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        "LAMBDA1":      [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        "LAMBDA2":      [0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00],
        "ALPHA":        [0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10],
        "U0":           [110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110., 110.],
        "W0COEFF":      [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        "WALL_TIME": 240,
        "CYCLE_TIME": 10,
        "CHECKPOINT_TIME": 1200,
        "SUBJOBS_BUFFER_SIZE": 1.0,
        "PRODUCTION_STEPS": 10000,
        "PRNT_FREQUENCY":   10000,
        "TRJ_FREQUENCY":    10000,
        "CM_KF": 25.00,
        "CM_TOL": 5,
        "POSRE_FORCE_CONSTANT": 25.0,
        "POSRE_TOLERANCE": 3.5,
        "ALIGN_KF_SEP": 2.5,
        "ALIGN_K_THETA": 25.0,
        "ALIGN_K_PSI":   25.0,
        "UMAX": 200.00,
        "ACORE": 0.062500,
        "UBCORE": 100.0,
        "FRICTION_COEFF": 0.500000,
        "TIME_STEP": 0.004,
        "VERBOSE": False,

  }

#job basename
options['BASENAME'] = basename

#displacement
options['DISPLACEMENT'] = Displacement_vector_in_Angstroms

#alignment atoms
ligand1_ref_atoms = [18, 14, 15] #@param {type:"raw"}
ligand2_ref_atoms = [ 9, 1, 6] #@param {type:"raw"}
options['ALIGN_LIGAND1_REF_ATOMS'] = [i for i in ligand1_ref_atoms]
options['ALIGN_LIGAND2_REF_ATOMS'] = [i for i in ligand2_ref_atoms]

#system pdb file
pdb = PDBFile(pdboutfile)
topology = pdb.topology
positions = pdb.positions

#LIGAND1_ATOMS (residue L1)
res1 = None
for chain in topology.chains():
  for residue in chain.residues():
    if residue.name == 'L1':
        res1 = residue
        break
ligand1_atoms = get_indexes_from_residue(res1)
options['LIGAND1_ATOMS'] = ligand1_atoms

#LIGAND2_ATOMS (residue L2)
res2 = None
for chain in topology.chains():
  for residue in chain.residues():
    if residue.name == 'L2':
        res2 = residue
        break
ligand2_atoms = get_indexes_from_residue(res2)
options['LIGAND2_ATOMS'] = ligand2_atoms

#cm atom of the ligand is the first reference atom
options['LIGAND1_CM_ATOMS'] = [ ligand1_atoms[ ligand1_ref_atoms[0] ] ]
options['LIGAND2_CM_ATOMS'] = [ ligand2_atoms[ ligand2_ref_atoms[0] ] ]

#CMs of the ligands
lig1cm = cm_from_indexes(topology, positions, options['LIGAND1_CM_ATOMS'] )
lig2cm = cm_from_indexes(topology, positions, options['LIGAND2_CM_ATOMS'] )

#CM atoms of the receptor
query = 'atom.residue.chain.id == "A" and atom.name == "CA"'
options['RCPT_CM_ATOMS'] = get_indexes_from_query(topology, query)

#offset
cmr = cm_from_indexes(topology, positions, options['RCPT_CM_ATOMS'])
options['LIGOFFSET'] = (lig1cm - cmr)/angstrom

#restrained atoms (same as RCPT_CM_ATOMS)
options['POS_RESTRAINED_ATOMS'] = options['RCPT_CM_ATOMS']

#working directory
options['WORKDIR'] = workDir

print("ATM RBFE SETTINGS:")
print(options)

rbfe_structprep(config_file = None, options = options)




In [ ]:
#@title ### **Run Relative Binding Free Energy Calculation**
rbfe_production(config_file = None, options = options)

### Analyisis: Binding free energy estimate

In [ ]:
from atom_openmm.uwham import calculate_uwham

ddG, ddG_std, dgbind1, dgbind2, samples = calculate_uwham(
          options['WORKDIR'], options['BASENAME'],
          mintimeid = 2,
          intermd=options['INTERMEDIATE'],
          lambda1=options['LAMBDA1'],
          lambda2=options['LAMBDA2'],
          alpha=options['ALPHA'],
          u0=options['U0'],
          w0=options['W0COEFF']
)
print(ddG, ddG_std, dgbind1, dgbind2, "kcal/mol", samples)

In [ ]:
#@title ### **Download the results as a zip file**
import zipfile
from google.colab import files

print("Downloading result files.")
filename = f'{workDir}.zip'

with zipfile.ZipFile(filename, 'w') as zip_file:
    dir_path = f'{workDir}'
    for root, directory, items in os.walk(dir_path):
        for item in items:
            path = os.path.join(root, item)
            zip_file.write(path, arcname=os.path.relpath(os.path.join(root, item), dir_path), compress_type=zipfile.ZIP_DEFLATED)

files.download(filename)

In [ ]:

#@title ### **Optionally, automatically disconnect the runtime when done**
from google.colab import runtime
import time
time.sleep(300)
runtime.unassign()